## Import the necessary packages

In [15]:
#!pip install xgboost

     |████████████████████████████████| 1.8 MB 9.3 MB/s eta 0:00:01


In [69]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import MinMaxScaler

from xgboost import XGBRegressor
from xgboost import plot_importance

from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import r2_score, mean_squared_error

import seaborn as sns

## Import and explore the data

In [5]:
df = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')

In [6]:
df.head()

,id,squareMeters,numberOfRooms,hasYard,hasPool,floors,cityCode,cityPartRange,numPrevOwners,made,isNewBuilt,hasStormProtector,basement,attic,garage,hasStorageRoom,hasGuestRoom,price
0,0,34291,24,1,0,47,35693,2,1,2000,0,1,8,5196,369,0,3,3436795.2
1,1,95145,60,0,1,60,34773,1,4,2000,0,1,729,4496,277,0,6,9519958.0
2,2,92661,45,1,1,62,45457,4,8,2020,1,1,7473,8953,245,1,9,9276448.1
3,3,97184,99,0,0,59,15113,1,1,2000,0,1,6424,8522,256,1,9,9725732.2
4,4,61752,100,0,0,57,64245,8,4,2018,1,0,7151,2786,863,0,7,6181908.8


### We have roughly 22k rows of data point and 18 columns potentially for our prediction model

In [7]:
df.shape

(22730, 18)

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22730 entries, 0 to 22729
Data columns (total 18 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 22730 non-null  int64  
 1   squareMeters       22730 non-null  int64  
 2   numberOfRooms      22730 non-null  int64  
 3   hasYard            22730 non-null  int64  
 4   hasPool            22730 non-null  int64  
 5   floors             22730 non-null  int64  
 6   cityCode           22730 non-null  int64  
 7   cityPartRange      22730 non-null  int64  
 8   numPrevOwners      22730 non-null  int64  
 9   made               22730 non-null  int64  
 10  isNewBuilt         22730 non-null  int64  
 11  hasStormProtector  22730 non-null  int64  
 12  basement           22730 non-null  int64  
 13  attic              22730 non-null  int64  
 14  garage             22730 non-null  int64  
 15  hasStorageRoom     22730 non-null  int64  
 16  hasGuestRoom       227

### Luckly the data is quite clean and doesn't have any missing values

In [9]:
df.isna().sum()

id                   0
squareMeters         0
numberOfRooms        0
hasYard              0
hasPool              0
floors               0
cityCode             0
cityPartRange        0
numPrevOwners        0
made                 0
isNewBuilt           0
hasStormProtector    0
basement             0
attic                0
garage               0
hasStorageRoom       0
hasGuestRoom         0
price                0
dtype: int64

In [10]:
df.describe()

,id,squareMeters,numberOfRooms,hasYard,hasPool,floors,cityCode,cityPartRange,numPrevOwners,made,isNewBuilt,hasStormProtector,basement,attic,garage,hasStorageRoom,hasGuestRoom,price
count,22730.000000,2.273000e+04,22730.000000,22730.000000,22730.000000,22730.000000,22730.000000,22730.000000,22730.000000,22730.000000,22730.000000,22730.000000,22730.000000,22730.000000,22730.000000,22730.000000,22730.000000,2.273000e+04
mean,11364.500000,4.658622e+04,48.241091,0.475891,0.452750,47.305983,50013.795996,5.585042,5.620766,2008.005059,0.467972,0.460009,5283.646634,5067.981698,530.469644,0.461681,5.153058,4.634457e+06
std,6561.730145,4.952124e+04,28.226428,0.499429,0.497773,47.777207,30006.637729,2.739533,2.713026,118.826777,0.498984,0.498409,3047.084412,3097.347939,274.840604,0.498540,3.055246,2.925163e+06
min,0.000000,8.900000e+01,1.000000,0.000000,0.000000,1.000000,3.000000,1.000000,1.000000,1990.000000,0.000000,0.000000,4.000000,1.000000,4.000000,0.000000,0.000000,1.031350e+04
25%,5682.250000,2.039275e+04,25.000000,0.000000,0.000000,25.000000,22936.000000,3.000000,3.000000,2000.000000,0.000000,0.000000,2977.250000,2599.000000,297.000000,0.000000,3.000000,2.041739e+06
50%,11364.500000,4.448400e+04,47.000000,0.000000,0.000000,45.000000,50414.000000,6.000000,6.000000,2006.000000,0.000000,0.000000,5359.000000,4977.000000,515.000000,0.000000,5.000000,4.450823e+06
75%,17046.750000,7.154700e+04,75.000000,1.000000,1.000000,69.000000,76291.000000,8.000000,8.000000,2014.000000,1.000000,1.000000,7746.000000,7652.000000,767.000000,1.000000,8.000000,7.159920e+06
max,22729.000000,6.071330e+06,100.000000,1.000000,1.000000,6000.000000,491100.000000,10.000000,10.000000,10000.000000,1.000000,1.000000,91992.000000,96381.000000,9017.000000,1.000000,10.000000,1.000428e+07


## Data cleaning

### Scale columns from 0 to 1

In [11]:
mms = MinMaxScaler()
df[['squareMeters','basement','attic','garage']] = mms.fit_transform(df[['squareMeters','basement','attic','garage']])

### Drop unnecessary columns

In [12]:
df.drop(['id','cityCode'], axis = 1, inplace = True)

### One-hot encoded the year made column

In [13]:
df['made'] = df['made'].astype(str)

df = pd.get_dummies(df, columns=['made'], drop_first = False)

### Change column name to reflect its context

In [14]:
df.rename({'hasGuestRoom':'numGuestRoom'}, axis = 1, inplace = True)

## Model training and evaluation

### We use GridSearchCV to find the most optimal model parameters to train the model

In [27]:
model = XGBRegressor()

# specify the parameters to mix the best model
cv_params = {'max_depth': [4,5,6,7,8], 
             'min_child_weight': [1,2,3,4,5],
             'learning_rate': [0.1, 0.2, 0.3],
             'n_estimators': [75, 100, 125]
             }

scoring = {'r2'}

xgb_cv = GridSearchCV(model, cv_params, scoring=scoring, cv=5, refit='r2')

# y-value
y = df['price']

# x-variables
X = df.drop(['price'], axis = 1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state = 10)

In [28]:
%%time
xgb_cv.fit(X_train, y_train)

CPU times: user 2h 15min 33s, sys: 8min 2s, total: 2h 23min 35s
Wall time: 19min 1s


GridSearchCV(cv=5,
             estimator=XGBRegressor(base_score=None, booster=None,
                                    callbacks=None, colsample_bylevel=None,
                                    colsample_bynode=None,
                                    colsample_bytree=None,
                                    early_stopping_rounds=None,
                                    enable_categorical=False, eval_metric=None,
                                    feature_types=None, gamma=None, gpu_id=None,
                                    grow_policy=None, importance_type=None,
                                    interaction_constraints=None,
                                    learning_rate=None, m...
                                    max_cat_to_onehot=None, max_delta_step=None,
                                    max_depth=None, max_leaves=None,
                                    min_child_weight=None, missing=nan,
                                    monotone_constraints=None, n_estim

### The most fit model parameter for having the best score, achieveing the high R2 score where 99% of the variables can explain the predicted housing price in the model

In [32]:
xgb_cv.best_params_

{'learning_rate': 0.2,
 'max_depth': 4,
 'min_child_weight': 2,
 'n_estimators': 75}

In [53]:
xgb_cv_predict = xgb_cv.predict(X_test)

In [73]:
r2 = r2_score(y_test, xgb_cv_predict)
print('R2 score:',r2)

R2 score: 0.9981342905747339


### It maybe a obvious finding but 97% of the model predicted housing price is contributed by the housing size, following up with the 0.92% for the exclusiveness of the neighbourhodd

In [63]:
column_list = list(X_train.columns)
feature_importance = xgb_cv.best_estimator_.feature_importances_.tolist()

feature_df = pd.DataFrame({'column_name':column_list, 'feature_importance':feature_importance})

In [64]:
feature_df = feature_df.sort_values(by='feature_importance', ascending=False)
feature_df['feature_importance'] = (feature_df['feature_importance']*100).round(2).astype(str) + '%'

print('Top 10 feature importance')
feature_df.head(10)

Top 10 feature importance


,column_name,feature_importance
0,squareMeters,97.43%
5,cityPartRange,0.92%
13,numGuestRoom,0.24%
9,basement,0.19%
11,garage,0.17%
30,made_2005,0.17%
25,made_2000,0.13%
1,numberOfRooms,0.1%
10,attic,0.09%
42,made_2017,0.09%
